## Tidal Debris from M33

In [6]:
import numpy as np
import astropy.units as u
from astropy.constants import G

# import plotting modules
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

# my modules
from ReadFile import Read

# edit CenterOfMass2 to Calculate COM of all components 
from CenterOfMass2 import CenterOfMass
# edit GalaxyMAss to Calculate Mass of all components 
from GalaxyMass import ComponentMass


In [8]:
def Mass_enclose(filename,radius):
# function to compute the mass of all particles less than the distance between two Galaxy COM position 
# Inputs:
#       filename: galaxyfile for input
#       radius: COM position difference in kpc 
# output: Mass in units of Msun. 
    
    # read in the file 
    time, total, data = Read(filename)
    
    # gather particles with the same type and sum up the mass
    mass = np.round(data[np.sqrt(data['x']**2+data['y']**2+data['z']**2) < radius]['m'].sum() * 1e10, 3)
    
    # return the mass 
    return mass


In [9]:
class Snap:
    """This is a class for each Snap with M31, MW, M33"""
    
    def __init__(self,Snap_number):
        # get the snap file name of MW M31 and M33
        # Inputs: Snapnumber
        
        self.Snap_number = Snap_number
        self.MWfilename = "VLowRes/" + 'MW' + "_VLowRes/" + 'MW' +  "_{:03d}".format(self.Snap_number) + ".txt"
        self.M31filename = "VLowRes/" + 'M31' + "_VLowRes/" + 'M31' +  "_{:03d}".format(self.Snap_number) + ".txt"
        self.M33filename = "VLowRes/" + 'M33' + "_VLowRes/" + 'M33' +  "_{:03d}".format(self.Snap_number) + ".txt"
        
    def COM(self):
        # Calculate the COM Position of MW, M31 and M33 
        
        MW_COM = CenterOfMass(self.MWfilename)
        M31_COM = CenterOfMass(self.M31filename)
        M33_COM = CenterOfMass(self.M31filename)
        
        # For MW and M31 delta = 0.1 and VelDec = 2
        # For M33 dalta = 0.1 and VelDec = 4
        MW_COM = MW_COM.COM_P(0.1,2)
        M31_COM = M31_COM.COM_P(0.1,2)
        M33_COM = M33_COM.COM_P(0.1,4)
        self.COMP_MW = COMP_MW.value
        self.COMP_M31 = COMP_M31.value
        self.COMP_M33 = COMP_M33.value
        
        return self.COMP_MW, self.COMP_M31, self.COMP_M33
    
    
    def COMdiff(self,galaxy1,galaxy2):
        # Calculate the COM Position distance of two Galaxy
        
        COMdiff = np.sqrt((galaxy1[0]-galaxy2[0])**2+(galaxy1[1]-galaxy2[1])**2+(galaxy1[2]-galaxy2[2])**2)        
        
        return COMdiff
    
    def Total_Mass(self,galaxy_name):
        # Calculate total Mass of each Galaxy 
        
        return ComponentMass(galaxy_name) 
    
    def Mass_enclose(self):
        # Calculate the Mass enclose less than the distance between two Galaxy COM Position
        
        #Calculate COM Positoon
        self.COMP_MW, self.COMP_M31, self.COMP_M33 = self.COM()
        
        #Calculate MW_M33 M31_M33 COMP difference
        self.MW_M33diff = self.COMdiff(self.COMP_MW,self.COMP_M33)
        self.M31_M33diff = self.COMdiff(self.COMP_M31,self.COMP_M33)
        
        #Calculate the mass inclose
        self.M31_enclose = Mass_enclose(self.M31filename, self.M31_M33diff)
        self.MW_enclose = Mass_enclose(self.MWfilename, self.MW_M33diff)
        
        return self.M31_enclose, self.MW_enclose
    
    def Rj(self)
        # Calculate the Jacobi Radius of M31-M33 and MW-M33 system
        # Rj = r*(Msat/Mhost(<r))**(1/3)
        # Msat is M33, Mhost is enclose mass of MW and M31
        self.MWmass = self.Total_Mass(self.MWfilename)
        self.M31mass = self.Total_Mass(self.M31filename)
        self.M33mass = self.Total_Mass(self.M33filename)
        
        self.COMP_MW, self.COMP_M31, self.COMP_M33 = self.COM()
        
        self.MW_M33diff = self.COMdiff(self.COMP_MW,self.COMP_M33)
        self.M31_M33diff = self.COMdiff(self.COMP_M31,self.COMP_M33)
        
        self.M31_enclose, self.MW_enclose = self.Mass_enclose()
        
        self.Rj_MW_M33 = self.MW_M33diff *(self.M33mass/self.MW_enclose)**(1/3)
        self.Rj_M31_M33 = self.M31_M33diff *(self.M33mass/self.M31_enclose)**(1/3)
        
        return self.Rj_M31_M33, self.Rj_MW_M33


In [35]:
def Select_tides_star(M33,Rj_MW_M33,Rj_M31_M33):
    """Select tides star of M33"""
    # Inputs:
    #      M33: each snap file of M33
    #      Rj_MW_M33: Jacobi Radius of MW and M33 system
    #      Rj_M1_M33: Jacobi Radius of M31 and M33 system
    # Outpur:
    #      tidal_star con
    time, total, data = Read(M33)
    MW_index = np.where(np.sqrt(data['x']**2+data['y']**2+data['z']**2) >= Rj_MW_M33)
    M31_index = np.where(np.sqrt(data['x']**2+data['y']**2+data['z']**2) >= Rj_M31_M33)
    total_index = M31_index
    for i in range(len(MW_index)):
        if MW_index[i] not in total_index:
            total_index = np.append(total_index,MW_index[i])
    tidal_star = data[total_index]
    
    return tidal_star

'VLowRes/MW_VLowRes/MW_111.txt'

5